**Импорт библиотек**

In [7]:
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import numpy as np

**Загрузка данных**

In [8]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
movies.drop(['genres'], axis=1, inplace=True)
ratings.drop(['timestamp'], axis=1, inplace=True)

**Составление матрицы предпочтений**

In [9]:
user_item_matrix = ratings.pivot_table(index='movieId', columns='userId', values='rating', fill_value=0)

**Фильтрация неактивных пользователей**

In [10]:
# Фильтруем данные (активные пользователи и фильмы)
users_votes = ratings.groupby('userId')['rating'].agg('count')
movies_votes = ratings.groupby('movieId')['rating'].agg('count')
user_mask = users_votes[users_votes > 50].index
movie_mask = movies_votes[movies_votes > 10].index
user_item_matrix = user_item_matrix.loc[movie_mask, user_mask]

**Разряжаем матрицу из-за наличия в большом кол-ве NaN значений**

In [11]:
# Преобразуем матрицу в разреженный формат
csr_data = csr_matrix(user_item_matrix.T)

**Строим модель ближайших соседей**

In [12]:
# Строим и обучаем модель
knn = NearestNeighbors(metric='cosine',
                       algorithm='brute',
                       n_neighbors=20,
                       n_jobs=-1)
knn.fit(csr_data)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=20)

**Определяем пользователя для которого требуется сделать рекомендацию**

In [13]:
# Ввод пользователя для которого нужно сделать рекомендацию
user_id_for_recommendation = 1 # Укажите id нужного пользователя
# Получаем список оцененных фильмов пользователем
user_ratings = user_item_matrix[user_id_for_recommendation].values.T

**Получаем индексы фильмов пользователя и строим средний вектор пользователя**

In [14]:
# Получаем индексы фильмов, которые пользователь оценил
rated_movie_indices = np.where(user_ratings != 0)[0]

if len(rated_movie_indices) == 0:
    print(f"Пользователь {user_id_for_recommendation} не оставил оценок. Нет рекомендаций.")
else:
    # Получаем индексы Id фильмов, которые оценил пользователь (Pandas Index)
    rated_movie_ids = user_item_matrix.index[rated_movie_indices]
    
    # Вычисляем номер строки(ID пользователя) в user_item_matrix (np.ndArray)
    rated_movie_indices_in_csr = np.array([user_item_matrix.columns.get_loc(user_id_for_recommendation)])

    # Вычисляем средний вектор пользователя и преобразуем его в массив NumPy
    user_vector = np.asarray(csr_data[rated_movie_indices_in_csr].mean(axis=0))

**Определяем рекомендации и длины их векторов**

In [15]:
# Строим рекомендации для усредненного вектора пользователя
recommendations = 10
distances, indices = knn.kneighbors(user_vector, n_neighbors=recommendations + 1)
indices_list = indices.squeeze().tolist()
distances_list = distances.squeeze().tolist()

indices_distances = list(zip(indices_list, distances_list))
indices_distances_sorted = sorted(indices_distances, key=lambda x: x[1], reverse=False)
indices_distances_sorted = indices_distances_sorted[1:]

In [16]:
indices_distances_sorted

[(229, 0.6047172774114625),
 (193, 0.609453847277913),
 (57, 0.6140971386896643),
 (165, 0.6182937397576205),
 (177, 0.6192237602240835),
 (34, 0.6209022250178893),
 (133, 0.6224025957331519),
 (10, 0.6255052165585675),
 (290, 0.628291406996312),
 (367, 0.6290824151242224)]

**Составляем список из 10 рекомендаций**

In [17]:
recom_list = []
for ind_dist in indices_distances_sorted:
    matrix_movie_id = user_item_matrix.index[ind_dist[0]]
    id = movies[movies['movieId'] == matrix_movie_id].index
    title = movies.iloc[id]['title'].values[0]
    dist = ind_dist[1]
    recom_list.append({'Title': title, 'Distance': dist})

recom_df = pd.DataFrame(recom_list, index=range(1, recommendations + 1))
print(f"Рекомендации для пользователя {user_id_for_recommendation}:")
print(recom_df)

Рекомендации для пользователя 1:
                                        Title  Distance
1                     Perfect World, A (1993)  0.604717
2                     Another Stakeout (1993)  0.609454
3                          Taxi Driver (1976)  0.614097
4                          Client, The (1994)  0.618294
5   Naked Gun 33 1/3: The Final Insult (1994)  0.619224
6                 Seven (a.k.a. Se7en) (1995)  0.620902
7               Miracle on 34th Street (1994)  0.622403
8                                Nixon (1995)  0.625505
9                     Oliver & Company (1988)  0.628291
10                  African Queen, The (1951)  0.629082
